In [114]:
import pandas as pd
import folium

In [115]:
line = pd.read_csv('./subway_line_long_lat.csv', encoding= 'cp949')
passenger = pd.read_csv('./subway_passenger_2022.csv',encoding = 'cp949')
passenger = passenger.drop(columns='등록일자')
passenger_time = pd.read_csv('./subway_time_data.csv', encoding='cp949')


In [116]:
line.info()
passenger.info()
passenger_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   연번       276 non-null    int64  
 1   호선       276 non-null    int64  
 2   역번호      276 non-null    object 
 3   역명       276 non-null    object 
 4   경도       276 non-null    float64
 5   위도       276 non-null    float64
 6   데이터기준일자  276 non-null    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 15.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219745 entries, 0 to 219744
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   사용일자    219745 non-null  int64 
 1   노선명     219745 non-null  object
 2   역명      219745 non-null  object
 3   승차총승객수  219745 non-null  int64 
 4   하차총승객수  219745 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 8.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56339 entries, 0 to 56

In [117]:
passenger.describe()

,사용일자,승차총승객수,하차총승객수
count,2.197450e+05,219745.000000,219745.000000
mean,2.022067e+07,10197.962802,10161.690696
std,3.448377e+02,10221.848266,10376.297181
min,2.022010e+07,1.000000,0.000000
25%,2.022040e+07,3472.000000,3341.000000
50%,2.022070e+07,7268.000000,7089.000000
75%,2.022100e+07,13467.000000,13308.000000
max,2.022123e+07,122543.000000,118237.000000


In [118]:
# 호선별 역사 위도, 경도 데이터 딕셔너리로 변환
line_data = line[['호선', '역명', '경도', '위도']]


for i in range(9) :
    globals()['line_{}'.format(i+1)] = line_data.query('호선 == {}'.format(i+1)).reset_index()


lat_dic = {}
long_dic = {}

#target_line 은 line_# 형식임
def transfer_line_select(target_line):
    range_num = len(target_line.index)
    for i in range(range_num):
        a = {target_line['역명'].iloc[i] : target_line['위도'][i]}
        lat_dic.update(a)
        b = {target_line['역명'].iloc[i] : target_line['경도'][i]}
        long_dic.update(b)

def transfer_line_all():
    for i in range(len(line_data.index)):
        a = {line_data['역명'].iloc[i] : line_data['위도'][i]}
        lat_dic.update(a)
        b = {line_data['역명'].iloc[i] : line_data['경도'][i]}
        long_dic.update(b)
transfer_line_all()

In [119]:
# 데이터 전처리 : passenger 데이터에 역별 위치 데이터 전처리

passenger['사용일자']= pd.to_datetime(passenger['사용일자'], format = '%Y%m%d')
passenger['요일'] = passenger['사용일자'].dt.day_name()
# 데이터 오류 관측
rdpg_day_sort = passenger.groupby(['요일','역명']).agg(n =('승차총승객수', 'mean')).sort_values(by = 'n')
error_data_detect = rdpg_day_sort.query('n == 1').reset_index().groupby(['요일', '역명']).mean()
error_station_list = list(set(error_data_detect.reset_index()['역명']))
print("결측데이터 :",error_station_list) 
#하차총승객수에는 에러데이터가 보이지 않음. 데이터 출처인 서울열린데이터광장에 따르면 수동 단말기로 취합하기 때문에 데이터오류가 있을 수 있다고 함.
#아래 데이터는 다른 출처의 데이터로 갈음함.

결측데이터 : ['신중동', '부천종합운동장', '부천시청', '부평구청', '상동', '까치울', '삼산체육관', '굴포천', '춘의']


In [120]:
#타 출처 데이터 전처리: 승객 시간별 승하차 데이터(2022)
passenger_time.head(10)
passenger_time['사용월'] = pd.to_datetime(passenger_time['사용월'], format = '%Y%m').dt.year
passenger_time =passenger_time.drop(columns=['작업일자','호선명'])


In [121]:
passenger_time.query('지하철역 == "삼산체육관" and 사용월 == 2021')


,사용월,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,07시-08시 하차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
7594,2021,삼산체육관,7,0,3408,452,10814,1847,25715,2855,...,553,4292,20,923,0,0,0,0,0,0
8203,2021,삼산체육관,14,0,3655,479,10985,1829,26594,2962,...,655,5493,20,1292,0,0,0,0,0,0
8812,2021,삼산체육관,18,0,3467,471,9774,1850,22973,3183,...,648,5082,15,746,0,0,0,0,0,0
9421,2021,삼산체육관,23,2,3151,425,9280,1607,21333,3186,...,571,4004,15,422,0,0,0,0,0,0
10028,2021,삼산체육관,19,0,3412,441,9463,1684,21723,2792,...,471,3611,19,381,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11850,2021,삼산체육관,3,0,3181,405,9576,1760,22266,3044,...,695,5168,12,708,0,0,0,0,0,0
12458,2021,삼산체육관,2,0,3109,381,10451,1753,25050,3333,...,675,4682,10,676,0,0,0,0,0,0
13066,2021,삼산체육관,7,0,3061,258,10301,1566,25225,2997,...,667,4648,6,616,0,0,0,0,0,0
13669,2021,삼산체육관,9,0,2267,192,7653,1266,18502,1505,...,412,2933,4,367,0,0,0,0,0,0


In [122]:
#좌표 찾기
import json
import requests

with open("./api.txt") as api:
    lines = api.readlines()
    api_key = lines[0].strip()
    
addr = '서울시 송파구 가락동 140' ##<- 주소를 입력
class catch_address():
    
    def addr_to_lat_long(addr):
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
        headers = {"Authorization": "KakaoAK " + api_key}
        result = json.loads(str(requests.get(url, headers=headers).text))
        match_first = result['documents'][0]['address']
        return float(match_first['x']), float(match_first['y'])
    
    def addr_lat(addr):
        a3 = catch_address.addr_to_lat_long(addr)
        a3_list = list(a3)
        return a3_list[0]
    
    def addr_long(addr):
        a3 = catch_address.addr_to_lat_long(addr)
        a3_list = list(a3)
        return a3_list[1]
    
    def search_lat_long(addr):
        return print(addr,'의 위도, 경도는 :', catch_address.addr_lat(addr), catch_address.addr_long(addr))
        
catch_address.search_lat_long(addr)

서울시 송파구 가락동 140 의 위도, 경도는 : 127.128117809357 37.4955429569204


In [123]:
map = folium.Map(location= [127.128117809357, 37.4955429569204], zoom_start=13, width = 750, height=500)

class data_mapping():
    
    def marker(lat, long, name, pop):
        return folium.Marker(location = [lat, long], popup= pop, tooltip = name )
    
    def circle_marker(lat, long, name, pop):
        radius_size = pop/2000 # 승하차 인원 비례
        return folium.CircleMarker(location=[lat, long], color = 'red', radius = radius_size)
    
    #def mapping_exe(): # 실행파일 작성

In [124]:
locate = []
for i in locate :
    lat = 1
    long = 2
    pop = 333333
    name = 45
    data_mapping.marker(lat, long, name, pop)

In [125]:
passenger['위도'] = passenger['역명']
passenger['경도'] = passenger['역명']

In [126]:
passenger['위도'] = passenger['위도'].replace(lat_dic)
passenger['경도'] = passenger['경도'].replace(long_dic)

In [127]:
passenger

,사용일자,노선명,역명,승차총승객수,하차총승객수,요일,위도,경도
0,2022-01-01,3호선,수서,7370,7076,Saturday,37.487507,127.101324
1,2022-01-01,3호선,학여울,461,473,Saturday,37.496757,127.070541
2,2022-01-01,3호선,대청,3224,2903,Saturday,37.493607,127.079526
3,2022-01-01,3호선,일원,3321,2803,Saturday,37.48389,127.08416
4,2022-01-01,경원선,창동,1,0,Saturday,37.652993,127.046746
...,...,...,...,...,...,...,...,...
219740,2022-12-31,4호선,신용산,11025,12099,Saturday,37.52919,126.96858
219741,2022-12-31,경부선,석수,5643,5147,Saturday,석수,석수
219742,2022-12-31,경부선,금천구청,7519,7426,Saturday,금천구청,금천구청
219743,2022-12-31,경부선,가산디지털단지,5579,7299,Saturday,37.480376,126.882704


In [128]:
passenger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219745 entries, 0 to 219744
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   사용일자    219745 non-null  datetime64[ns]
 1   노선명     219745 non-null  object        
 2   역명      219745 non-null  object        
 3   승차총승객수  219745 non-null  int64         
 4   하차총승객수  219745 non-null  int64         
 5   요일      219745 non-null  object        
 6   위도      219745 non-null  object        
 7   경도      219745 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 13.4+ MB


In [129]:
passenger_sort = passenger.query('사용일자 == "2022-06-20"').reset_index()

In [130]:
passenger_sort['특정'] = passenger_sort['위도'].str.isdigit()

In [134]:
passenger_sort = passenger_sort.drop(columns='index')

In [135]:
passenger_sort

,사용일자,노선명,역명,승차총승객수,하차총승객수,요일,위도,경도,특정
0,2022-06-20,5호선,까치산,30468,28922,Monday,37.53181,126.846706,NaN
1,2022-06-20,5호선,신정(은행정),14956,12136,Monday,신정(은행정),신정(은행정),False
2,2022-06-20,5호선,목동,18110,19301,Monday,37.526088,126.864296,NaN
3,2022-06-20,5호선,오목교(목동운동장앞),22757,24204,Monday,오목교(목동운동장앞),오목교(목동운동장앞),False
4,2022-06-20,5호선,양평,7967,8025,Monday,37.525614,126.886177,NaN
...,...,...,...,...,...,...,...,...,...
600,2022-06-20,수인선,남동인더스파크,2326,2690,Monday,남동인더스파크,남동인더스파크,False
601,2022-06-20,수인선,호구포,4383,4565,Monday,호구포,호구포,False
602,2022-06-20,수인선,인천논현,5890,6358,Monday,인천논현,인천논현,False
603,2022-06-20,수인선,소래포구,6182,5702,Monday,소래포구,소래포구,False
